In [1]:
import os
os.chdir('C:/projects/itmo/higher_education_analyzer/')

In [145]:
import pandas, json
from pprint import pprint
import sklearn
import seaborn
import numpy
import itertools
import fasttext
import fasttext.util
from sklearn.cluster import DBSCAN as dbscan
from collections import Counter
import re
from utils import *

In [3]:
fasttext.util.download_model('ru', if_exists='ignore') 
ft = fasttext.load_model('cc.ru.300.bin')

In [4]:
data = []
with open('data/vacancies.json') as f:
    data = json.load(f)

In [5]:
pprint(data[0])

{'accept_handicapped': False,
 'accept_incomplete_resumes': False,
 'accept_kids': False,
 'accept_temporary': False,
 'allow_messages': False,
 'alternate_url': 'https://hh.ru/vacancy/67164932',
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=67164932',
 'archived': False,
 'area': {'id': '2759',
          'name': 'Ташкент',
          'url': 'https://api.hh.ru/areas/2759'},
 'billing_type': {'id': 'premium', 'name': 'Премиум'},
 'branded_description': None,
 'code': 'Преподаватель (МЕНТОР)',
 'contacts': None,
 'created_at': '2022-06-24T15:05:17+0300',
 'department': None,
 'description': '<p>В IT академию «ASTRUM» требуются IT-СПЕЦИАЛИСТЫ (МЕНТОР) с '
                'навыками работы преподавания</p> '
                '<p><strong>Обязанности:</strong></p> <ul> <li>Организует '
                'поддержку по учебной платформе для слушателей</li> '
                '<li>Координирует слушателей по своему направлению.</li> '
                '<li>Проводит дисципл

In [6]:
def normalize_skill(s):
    # return s.lower().replace(',', '').replace('.', '')
    return re.sub(r'[\.\,]+|\s{2,}', '', s).lower()

def extract_skills(vacancies):
    return list(set(normalize_skill(skill['name']) for skill in itertools.chain.from_iterable(vac['key_skills'] for vac in vacancies)))

In [7]:
all_skills = extract_skills(data)
print(f'общее число для разных навыков: {len(all_skills)}')

общее число для разных навыков: 3157


In [18]:
skill_vecs = [ft.get_word_vector(word) for word in all_skills]

In [19]:
clusters = dbscan(eps=0.105, n_jobs=-1, min_samples=2)
clusters.fit(skill_vecs)
print('uniques:', Counter(clusters.labels_))

uniques: Counter({-1: 3145, 0: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 2})


In [16]:
skill2labels = sorted([skill2label for skill2label in zip(all_skills, clusters.labels_) if skill2label[1] > -1], key=lambda item: item[1])

In [17]:
pprint([(key, [v[0] for v in value]) for key, value in itertools.groupby(skill2labels, key=lambda item: item[1])])

[(0, ['техническая грамотность', 'алгоритмическая грамотность']),
 (1, ['социологические исследования', 'аналитические исследования']),
 (2, ['юридическая поддержка', 'техническая поддержка']),
 (3, ['аналитический склад ума', 'математический склад ума']),
 (4,
  ['формирование индивидуальных планов развития',
   'построение индивидуальных планов развития']),
 (5,
  ['исполнение личных поручений руководителя',
   'исполнение поручений руководителя']),
 (6, ['линейное программирование', 'асинхронное программирование']),
 (7,
  ['финансовое планирование',
   'маркетинговое планирование',
   'кадровое планирование']),
 (8, ['экологическая безопасность', 'информационная безопасность']),
 (9, ['качественные исследования', 'количественные исследования']),
 (10, ['написание аналитических отчетов', 'написание аналитических текстов']),
 (11, ['профессиональный пользователь пк', 'продвинутый пользователь пк']),
 (12, ['интеграционное тестирование', 'регрессионное тестирование']),
 (13,
  ['умени

In [21]:
skill_vecs = [ft.get_sentence_vector(word) for word in all_skills]

In [39]:
clusters = dbscan(eps=0.4, n_jobs=-1, min_samples=2)
clusters.fit(skill_vecs)
print('uniques:', Counter(clusters.labels_))

uniques: Counter({-1: 2650, 20: 47, 9: 19, 12: 11, 48: 9, 23: 6, 44: 6, 59: 6, 128: 6, 32: 5, 35: 5, 41: 5, 69: 5, 83: 5, 47: 4, 54: 4, 55: 4, 67: 4, 82: 4, 94: 4, 135: 4, 0: 3, 3: 3, 5: 3, 7: 3, 8: 3, 13: 3, 14: 3, 16: 3, 17: 3, 30: 3, 33: 3, 37: 3, 38: 3, 42: 3, 43: 3, 50: 3, 52: 3, 62: 3, 66: 3, 72: 3, 76: 3, 89: 3, 90: 3, 95: 3, 98: 3, 99: 3, 102: 3, 103: 3, 110: 3, 115: 3, 126: 3, 130: 3, 136: 3, 143: 3, 150: 3, 154: 3, 160: 3, 169: 3, 1: 2, 2: 2, 4: 2, 6: 2, 10: 2, 11: 2, 15: 2, 18: 2, 19: 2, 21: 2, 22: 2, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2, 31: 2, 34: 2, 36: 2, 39: 2, 40: 2, 45: 2, 46: 2, 49: 2, 51: 2, 53: 2, 56: 2, 57: 2, 58: 2, 60: 2, 61: 2, 63: 2, 64: 2, 65: 2, 68: 2, 70: 2, 71: 2, 73: 2, 74: 2, 75: 2, 77: 2, 78: 2, 79: 2, 80: 2, 81: 2, 84: 2, 85: 2, 86: 2, 87: 2, 88: 2, 91: 2, 92: 2, 93: 2, 96: 2, 97: 2, 100: 2, 101: 2, 104: 2, 105: 2, 106: 2, 107: 2, 108: 2, 109: 2, 111: 2, 112: 2, 113: 2, 114: 2, 116: 2, 117: 2, 118: 2, 119: 2, 120: 2, 121: 2, 122: 2, 123: 2, 124: 2,

In [40]:
skill2labels = sorted([skill2label for skill2label in zip(all_skills, clusters.labels_) if skill2label[1] > -1], key=lambda item: item[1])

In [41]:
pprint([(key, [v[0] for v in value]) for key, value in itertools.groupby(skill2labels, key=lambda item: item[1])])

[(0,
  ['знание китайского языка',
   'знание английского языка',
   'знание английского']),
 (1, ['решение проблем', 'эффективное решение проблем']),
 (2, ['первичная документация', 'первичная бухгалтерская документация']),
 (3, ['финансовый анализ', 'экономический анализ', 'инвестиционный анализ']),
 (4, ['1с программирование', 'программирование 1с']),
 (5,
  ['разработка документации',
   'разработка технической документации',
   'разработка проектной документации']),
 (6, ['php 73', 'php 74']),
 (7,
  ['обновление конфигурации 1с',
   'конфигурации 1с',
   'создание конфигурации 1с']),
 (8, ['full stack', 'javascript full stack', 'full stack developer']),
 (9,
  ['api ms windows',
   'субд microsoft sql server',
   'ms sql server 2012/2016',
   'windows server 2003',
   'ms windows server 2016',
   'microsoft windows server 2016',
   'ms exchange',
   'с# wpf wcf ms sql server',
   'windows api',
   'windows server 2008 r2',
   'windows server',
   'ms exchange server',
   'ms sql 

In [43]:
data[0]['description']

'<p>В IT академию «ASTRUM» требуются IT-СПЕЦИАЛИСТЫ (МЕНТОР) с навыками работы преподавания</p> <p><strong>Обязанности:</strong></p> <ul> <li>Организует поддержку по учебной платформе для слушателей</li> <li>Координирует слушателей по своему направлению.</li> <li>Проводит дисциплинарные беседы со слушателями по внутреннему распорядку заведения.</li> <li>Общается со слушателями и передает проблемы и предложения Руководителю учебного отдела.</li> <li>Управляет слушателями в социальной деятельности заведения, координирует и информирует.</li> <li>Участвует в разработке и доработке учебного процесса заведения, а также участвует во всех мероприятиях</li> <li>Постоянно совершенствует свои навыки изучением материалов и выполнением практических заданий на учебной платформе</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <p>- профильное высшее/средне-специальное образование</p> <p>- знание русского, узбекского, английского языков</p> <p>для <strong>Software engineer ментора</strong>:</p>

In [44]:
from bs4 import BeautifulSoup

In [151]:
def prepare_text(html):
    soup = BeautifulSoup(html)
    text = soup.get_text().lower()
    text = re.sub(r'/|\\|\(|\)|-|,|:|\.|\d', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()
    
prepare_text(data[0]['description'])

'в it академию «astrum» требуются it специалисты ментор с навыками работы преподавания обязанности организует поддержку по учебной платформе для слушателей координирует слушателей по своему направлению проводит дисциплинарные беседы со слушателями по внутреннему распорядку заведения общается со слушателями и передает проблемы и предложения руководителю учебного отдела управляет слушателями в социальной деятельности заведения координирует и информирует участвует в разработке и доработке учебного процесса заведения а также участвует во всех мероприятиях постоянно совершенствует свои навыки изучением материалов и выполнением практических заданий на учебной платформе требования профильное высшее средне специальное образование знание русского узбекского английского языков для software engineer ментора знание html css javascript c c++ для data science ментора знание html css javascript python pandas numpy matplotlib для fullstack ментора знание html css flexbox grid javascript ruby react js 

In [152]:
for vac in data:
    vac['description_norm'] = prepare_text(vac['description'])

In [153]:
data[0]

{'id': '67164932',
 'premium': True,
 'billing_type': {'id': 'premium', 'name': 'Премиум'},
 'relations': [],
 'name': 'IT-Специалисты (ментор) с навыками работы преподавания',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '2759',
  'name': 'Ташкент',
  'url': 'https://api.hh.ru/areas/2759'},
 'allow_messages': False,
 'experience': {'id': 'between3And6', 'name': 'От 3 до 6 лет'},
 'department': None,
 'contacts': None,
 'description': '<p>В IT академию «ASTRUM» требуются IT-СПЕЦИАЛИСТЫ (МЕНТОР) с навыками работы преподавания</p> <p><strong>Обязанности:</strong></p> <ul> <li>Организует поддержку по учебной платформе для слушателей</li> <li>Координирует слушателей по своему направлению.</li> <li>Проводит дисциплинарные беседы со слушателями по внутреннему распорядку заведения.</li> <li>Общается со слушателями и передает проблемы и предложения Руководителю учебного отдела.</li> <li>Управляет слушателями в социальной деятельности заведения, координирует и

In [102]:
import gensim
import nltk
from gensim import corpora
from gensim.models import LdaModel

In [206]:
nltk.download('stopwords')
ru_stops = set(nltk.corpus.stopwords.words('russian'))
en_stops = set(nltk.corpus.stopwords.words('english'))
my_stops = set(readlines('stopwords.txt'))
stops = ru_stops | en_stops | my_stops

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avdosev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [207]:
def preprocess_text(text):
    return [word for word in text.split(' ') if (word not in stops) and word.isalpha()]

In [208]:
texts = [preprocess_text(vac['description_norm']) for vac in data]

In [209]:
texts[0]

['академию',
 'требуются',
 'специалисты',
 'ментор',
 'работы',
 'преподавания',
 'организует',
 'поддержку',
 'учебной',
 'платформе',
 'слушателей',
 'координирует',
 'слушателей',
 'своему',
 'направлению',
 'проводит',
 'дисциплинарные',
 'беседы',
 'слушателями',
 'внутреннему',
 'распорядку',
 'заведения',
 'общается',
 'слушателями',
 'передает',
 'проблемы',
 'предложения',
 'руководителю',
 'учебного',
 'отдела',
 'управляет',
 'слушателями',
 'социальной',
 'деятельности',
 'заведения',
 'координирует',
 'информирует',
 'участвует',
 'доработке',
 'учебного',
 'процесса',
 'заведения',
 'участвует',
 'мероприятиях',
 'постоянно',
 'совершенствует',
 'изучением',
 'материалов',
 'выполнением',
 'практических',
 'заданий',
 'учебной',
 'платформе',
 'требования',
 'профильное',
 'средне',
 'специальное',
 'образование',
 'русского',
 'узбекского',
 'английского',
 'языков',
 'software',
 'engineer',
 'ментора',
 'html',
 'css',
 'javascript',
 'c',
 'data',
 'science',
 'менто

In [210]:
dictionary = corpora.Dictionary(texts)

In [211]:
print(dictionary)

Dictionary<47058 unique tokens: ['api', 'c', 'cloud', 'css', 'data']...>


In [212]:
dictionary.filter_extremes(no_below=5, no_above=0.50, keep_n=100000)

In [213]:
print(dictionary)

Dictionary<14962 unique tokens: ['api', 'c', 'cloud', 'css', 'data']...>


In [214]:
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]

In [218]:
lda_model = gensim.models.LdaModel(bow_corpus,
                                   id2word=dictionary,
                                   num_topics=15,
                                   offset=3,
                                   random_state=100,
                                   passes=10,
                                   alpha='auto',
                                   eta="auto",
                                   per_word_topics=True)

In [216]:
import pyLDAvis
import pyLDAvis.gensim_models

In [219]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
vis

C:\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.138097 -0.015421       1        1  42.256370
4      0.174250  0.089133       2        1  11.181523
10     0.182736  0.184363       3        1  10.753675
6     -0.387477  0.133575       4        1   9.590483
0      0.149873  0.193796       5        1   7.741405
12     0.098420  0.119041       6        1   6.287162
8     -0.396123  0.100174       7        1   5.401392
13     0.040573 -0.005003       8        1   2.653491
14     0.018499 -0.101876       9        1   0.887433
7      0.025891 -0.075029      10        1   0.771414
5      0.017945 -0.098501      11        1   0.684165
1     -0.006418 -0.115563      12        1   0.532512
11    -0.040056 -0.154341      13        1   0.478219
9      0.002252 -0.107935      14        1   0.457393
2     -0.018459 -0.146414      15        1   0.323362, topic_info=             Term         Freq        Total Category  logprob  loglift
4            data  1450.000000  1450.000000  Default  30.0000  30.0000
204        систем  1632.000000  1632.000000  Default  29.0000  29.0000
12             js   851.000000   851.000000  Default  28.0000  28.0000
88            sql  1434.000000  1434.000000  Default  27.0000  27.0000
560    технологий   975.000000   975.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
3704     завтраки     6.958184    35.112326  Topic15  -5.5916   4.1155
2171      проекта     7.664599   610.143397  Topic15  -5.4949   1.3571
2907  компенсация     7.534618   426.687948  Topic15  -5.5120   1.6976
1976        важно     7.230019   346.038442  Topic15  -5.5532   1.8658
1862  возможности     7.131704   479.404780  Topic15  -5.5669   1.5262

[1117 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
243        4  0.414582        ability
243        7  0.581043        ability
2257       4  0.978741      according
2257       7  0.015536      according
4291       4  0.959939        achieve
...      ...       ...            ...
3612       2  0.941454    эффективная
1843       1  0.201264  эффективности
1843       2  0.792072  эффективности
13928     13  0.883220           юмор
14040     11  0.905417           янао

[2981 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 11, 7, 1, 13, 9, 14, 15, 8, 6, 2, 12, 10, 3])